## Exploration and Attack

In [6]:
# Default is 16mhz, max for both compiled and max is 16.6mhz; it's not stable at higher frequencies
# Compiled clock rate = what the firmware uses when calculating the baud rates
COMPILED_CLK_FREQ  = 16000000
# What we use for the clkgen
ACTUAL_CLKGEN_FREQ = 16000000
# Baud rate for the firmware is currently hard-coded to 115200
COMPILED_BAUD_RATE = 115200
 # The scope adc doesn't lock when using ext_clock due to the N76E003's internal oscillator having high variance, so it's recommended set this to 1 to have the N76E003 use CLKIN as the clock source
USE_EXTERNAL_CLOCK = 1
# SS_VER_1_x is only supported when using no crypto targets; not enough memory.
SS_VER = ""
PLATFORM = "CW308_N76E003"
# Only crypto target supported is TINYAES128C
CRYPTO_TARGET = "NONE"


In [7]:
import math
import sys
import binascii

import time
import logging
import os
from collections import namedtuple
from chipwhisperer.capture.targets import TargetTypes, SimpleSerial
import numpy as np
import chipwhisperer as cw
from tqdm import trange
from NormalSerial import NormalSerial

def set_gpio(scope: cw.scopes.OpenADC):
	scope.io.nrst = None
	scope.io.pdic = None
	scope.io.pdid = None
	scope.io.tio3 = None

	scope.io.hs2 = None
	scope.io.tio1 = "serial_rx"
	scope.io.tio2 = "serial_tx"
	scope.io.tio3 = "high_z"
	scope.io.tio4 = "high_z"

def fb_setup(scope: cw.scopes.OpenADC):

	#Initial Setup 
	set_gpio(scope)
	# For this, we're using tio3 as the trigger
	scope.trigger.triggers = "tio3"
	
	scope.clock.clkgen_freq = ACTUAL_CLKGEN_FREQ
	# scope.clock.clkgen_div = 52
	# scope.clock.clkgen_mul = 9
	scope.adc.basic_mode = "rising_edge"
	scope.adc.samples = 22000
	scope.adc.offset = 0
	scope.clock.adc_src = "clkgen_x4"
	scope.glitch.trigger_src = "ext_single"
	scope.glitch.clk_src = "clkgen"
	scope.glitch.output = "glitch_only"

def calc_UART0_timer3_actual_baud_rate(compiled_fsys, baud_rate):
    SMOD = 1
    TH3_PRESCALE = 1
	# RH, RL values for the serial timer
    RH_RL = (65536 - round(math.floor(compiled_fsys/16)/baud_rate))
    timer3_actual_baud_rate = 2**SMOD/32 * compiled_fsys/(TH3_PRESCALE*(65536-RH_RL))
    return timer3_actual_baud_rate

def target_setup(target: NormalSerial):
	# TODO: remove this in the final version
	# earlier versions of the boards (pre 1.2) had these swapped, and that is what I'm currently testing with
	# set `_using_earlier_board` to `False` if you are using the latest version of the board
	_using_earlier_board = os.environ.get("USING_EARLIER_BOARD", False)
	if _using_earlier_board:
		print("USING EARLIER BOARD WITH SWAPPED TX AND RX!!!")
		scope.io.tio1 = "serial_tx"
		scope.io.tio2 = "serial_rx"
	else:
		print("SERIAL LINES: scope.io.tio1 = serial_rx, scope.io.tio2 = serial_tx")
		scope.io.tio1 = "serial_rx"
		scope.io.tio2 = "serial_tx"
	calced_baud_rate = calc_UART0_timer3_actual_baud_rate(COMPILED_CLK_FREQ, COMPILED_BAUD_RATE)
	print("calculated baud rate with compiled clock {}: ".format(COMPILED_CLK_FREQ), calced_baud_rate)
	target_baud_rate = calced_baud_rate * (ACTUAL_CLKGEN_FREQ / COMPILED_CLK_FREQ)
	print("Target baud rate: ", round(target_baud_rate))
	target.baud = round(target_baud_rate)

scope: cw.scopes.OpenADC = None
target: NormalSerial = None

def reconnect(type = NormalSerial):
	global scope
	global target
	try:
		scope.dis()
		target.dis()
	except:
		pass
	PLATFORM = "CW308_N76E003"
	%run "devsetup.ipynb"
	scope.default_setup(False)
	time.sleep(1)
	fb_setup(scope)
	target_setup(target)


In [8]:
import subprocess
import platform

MAKE_COMMAND = "make" if platform.system() != "Darwin" else "gmake"

def make_image(fw_dir:str, target_name:str = ""):
	# get the last part of fw_dir
	fw_base = target_name if target_name else os.path.basename(fw_dir)
	print(subprocess.check_output([MAKE_COMMAND, "clean"], 
					cwd=fw_dir).decode("utf-8"))	
	args = [
            MAKE_COMMAND,
            "PLATFORM={}".format(PLATFORM),
            "USE_EXTERNAL_CLOCK={}".format(USE_EXTERNAL_CLOCK), 
            "CRYPTO_TARGET={}".format(CRYPTO_TARGET), 
            "SS_VER={}".format(SS_VER), 
            "F_CPU={}".format(COMPILED_CLK_FREQ),
			"BAUD_RATE={}".format(COMPILED_BAUD_RATE),
            "-j"
            ]
	print(subprocess.check_output(args, cwd=fw_dir).decode("utf-8"))
	fw_path = os.path.join(fw_dir, "{}-{}.hex".format(fw_base, PLATFORM))
	return fw_path


In [9]:
from typing import Optional
from chipwhisperer import glitch_logger, target_logger
from NormalSerial import NormalSerial
from TestSetup import TestOptions, TestSetupTemplate, TestResult
from glitch_params import GlitchControllerParams
from nuvoprogpy.config import N76E003_DEVID
from nuvoprogpy.nuvo51icpy import Nuvo51ICP, ConfigFlags
from chipwhisperer.capture.api.programmers import Programmer
from programmer_n76_icp import N76ICPProgrammer, newaeUSBICPLib

NORMAL_I_VAL = 50
NORMAL_J_VAL = 50
NORMAL_CNT_VAL = NORMAL_I_VAL * NORMAL_J_VAL
BYTES_READ = 6
class NuvoConfigTest(TestSetupTemplate):
	_name = "NuvoConfigTest"
	def __init__(self, 
			  scope: cw.scopes.ScopeTypes, 
			  target: cw.targets.TargetTypes, 
			  programmer_type: Optional[type[Programmer]], 
			  glitch_params: GlitchControllerParams, 
			  options: TestOptions = None,
			  config_to_test: ConfigFlags = None):
		super().__init__(scope, target, programmer_type, glitch_params, options)
		self._debug = True
		self._print_read_write_times = False
		self.nuvo_config = config_to_test
		self.programmer_args = {"config_bytes": self.nuvo_config.to_bytes()}
		self.nuvo = None

	def _check_config(self, config: ConfigFlags):
		old_config = self.nuvo.read_config()
		if old_config.is_locked():
			self.logger.info("Device is locked, re-flashing image...")
			data = N76ICPProgrammer.convert_to_bin(self.fw_image_path)
			self.nuvo.program_all(data, config = config)
			self.nuvo.close()
			self.nuvo.init()
			old_config = self.nuvo.read_config()
		if str(old_config) == str(config):
			self.logger.info("Config matches, not setting...")
			return True
		self.logger.info("Setting config...")
		self.nuvo.program_config(config)
		self.nuvo.close()
		self.nuvo.init()
		new_config = self.nuvo.read_config()
		if str(new_config) != str(config):
			raise IOError("Failed to set config")

	
	def prep_run(self) -> bool:
		self._saved_io_nrst = self.scope.io.nrst
		self._saved_io_pdic = self.scope.io.pdic
		self._saved_io_pdid = self.scope.io.pdid
		self._saved_io_tio3 = self.scope.io.tio3
		self.scope.io.nrst = None
		self.scope.io.pdic = None
		self.scope.io.pdid = None
		self.scope.io.tio3 = None
		config = self.nuvo_config
		self.nuvo = Nuvo51ICP(library=newaeUSBICPLib(self.scope))
		self.nuvo.init()
		try:
			self._check_config(config)
		except Exception as e:
			self.logger.error("FAILED TO SET CONFIG!!!")
			raise e

		try:
			self.nuvo.close()
			self.nuvo = None
			self.logger.info("Waiting 10 seconds....")
			time.sleep(10)
			self.nuvo = Nuvo51ICP(library=newaeUSBICPLib(self.scope))
			self.nuvo.init()
		except Exception as e:
			self.logger.error("FAILED TO INIT NUVO!!!")
			raise e
		return True

	def after_run(self) -> None:
		if self.nuvo is not None:
			self.nuvo.close()
		self.scope.io.nrst = self._saved_io_nrst
		self.scope.io.pdic = self._saved_io_pdic
		self.scope.io.pdid = self._saved_io_pdid
		self.scope.io.tio3 = self._saved_io_tio3

	def iter_run(self) -> bool:
		# check if we are still in nuvo.icp mode
		dev_id = self.nuvo.get_device_id()
		if dev_id != N76E003_DEVID:
			self.logger.warning("Device not in nuvo.icp mode, resetting...")
			# self.nuvo.icp.exit()
			self.nuvo.close()
			
			time.sleep(1)
			self.nuvo.init()
			# self.nuvo.icp.entry()
			time.sleep(1)
			dev_id = self.nuvo.get_device_id()
		if dev_id != N76E003_DEVID:
			raise IOError("Device not in nuvo.icp mode")
		self.nuvo.icp.exit()
		time.sleep(0.1)
		self.nuvo.reenter_icp()
		self.nuvo.icp.reentry_glitch()
		return True

	def get_data(self):
		device_id: int = 0
		device_id = self.nuvo.get_device_id()
		
		return device_id

	def check_result(self, device_id) -> TestResult:
		if device_id != N76E003_DEVID:
			return TestResult.reset
		return TestResult.normal

In [10]:
from glitch_params import GlitchControllerParams

def device_reset():
	scope.io.nrst = 'low'
	time.sleep(0.05)
	scope.io.nrst = None
	time.sleep(0.05)

def reboot_flush():
	target.flush()
	device_reset()
	target.flush()


# The adc sometimes won't lock on 16600000 Hz, so we need to test various values above and below and see if it locks
def test_locking_clock_freq(scope: cw.scopes.OpenADC):
	for j in range(-1, 2):
		if j == 0:
			continue
		for i in range(0, 20):
			set_freq = ACTUAL_CLKGEN_FREQ + (i * 4000 * j)
			scope.clock.clkgen_freq = set_freq
			time.sleep(1)
			print("Testing clock frequency %.2f (actual: %.2f), div %d, mul %d...." % (set_freq, scope.clock.clkgen_freq, scope.clock.clkgen_div, scope.clock.clkgen_mul))
			if scope.clock.adc_locked:
				print("LOCKED")
				print("Attempting to arm glitcher...")
				
				scope.arm()
				print("Manually triggering glitcher...")
				
				scope.io.tio4 = True
				scope.io.tio4 = False
				time.sleep(1)
				if scope.capture():
					print("SCOPE TIMED OUT!!!")
				else:
					print("Successful ")
			else:
				print("not locked")

def capture_run(rom_dir: str = "", name = "", config= ConfigFlags):
	time.sleep(1)
	params = GlitchControllerParams(1, 1, 1, 1)
	options = TestOptions()
	options.fw_image_path = make_image(rom_dir)
	test = NuvoConfigTest(scope, target, N76ICPProgrammer, params, options, config)

	data = test.capture_sequence(5, name)
	mean_data = np.mean(data, axis = 0)
	return data, mean_data

reconnect()
# test_locking_clock_freq(scope)
base_fw_dir = "./numicro8051/blink-forever"
# get current script dir

data, mean_data = capture_run(base_fw_dir, "read-config-FF-FF-FF-FF-FF", ConfigFlags())

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.


OSError: Could not find ChipWhisperer. Is it connected?

OSError: Could not find ChipWhisperer. Is it connected?

In [ ]:
# INNER_LOOP_START = 0
# INNER_LOOP_END = 0
# WIDTH_MIN = 1
# WIDTH_MAX = 1
# WIDTH_STEP = 0.1
# width_range = [WIDTH_MIN, WIDTH_MAX, WIDTH_STEP]
# offset_range = [0, 0, 1]
# ext_offset_range = [INNER_LOOP_START, INNER_LOOP_END, 1]
# repeat_range = 1

voltage drop at sample 145, 11288.  
About 11145-11250 samples between voltage drops when led toggle is on outer loop

